In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np
import pickle
import csv

#Archivo por leer
data_path = "./weatherAUS.csv"
#Donde guardar los archivos
saved_path = "C:/Users/diego/Desktop/p4_k_folds"
num_col = 22
#porcentaje de datos de entrenamiento
n_train = 0.8

class validation_set:
	def __init__(self, X_train, y_train, X_test, y_test):
		self.X_train = X_train
		self.y_train = y_train
		self.X_test = X_test
		self.y_test = y_test


In [14]:
class test_set:
	def __init__(self, X_test, y_test):
		self.X_test = X_test
		self.y_test = y_test

class data_set:
	def __init__(self, validation_set, test_set, col_names):
		self.validation_set = validation_set
		self.test_set = test_set
		self.col_names = col_names

def generate_train_test(file_name, k):
	# ~ pd.options.display.max_colwidth = 200		#Define el acho de las columnas (ancho máximo por default 50 caracteres)		
	#Lee el corpus original del archivo de entrada y lo pasa a un DataFrame
	df = pd.read_csv(file_name, sep=',', usecols = range(num_col + 1),  engine='python')	
	X = df.drop('RainTomorrow',axis=1).values    						#corpus sin etiquetas 
	y = df['RainTomorrow'].values 										#etiquetas
	
	col_names = df.columns.values						
	
	#Separa corpus en conjunto de entrenamiento y prueba
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - n_train, shuffle = True, random_state = 0 )			
	
	#Crea pliegues para la validación cruzada
	validation_sets = []
	kf = KFold(n_splits = k)
	c = 0 #Pliegue
	for train_index, test_index in kf.split(X_train):
		c = c + 1
		X_train_v, X_test_v = X_train[train_index], X_train[test_index]
		y_train_v, y_test_v = y_train[train_index], y_train[test_index]
		#Agrega el pliegue creado a la lista
		validation_sets.append(validation_set(X_train_v, y_train_v, X_test_v, y_test_v))		
	
	#Almacena el conjunto de prueba
	my_test_set = test_set(X_test, y_test)	
	
	#Guarda el dataset con los pliegues del conjunto de validación y el conjunto de pruebas
	my_data_set = data_set(validation_sets, my_test_set, col_names) 
	return (my_data_set)

In [15]:
def createFiles(my_data_set, k):        
    c = 0 #pliegues  
    x_names = my_data_set.col_names[:-1]
    y_names = my_data_set.col_names[-1:]    

    for n in my_data_set.validation_set:        
        c = c + 1        
        with open("./cross_validation_files/data_validation_train_{}_{}.csv".format(str(k), str(c)), 'w', newline='', encoding="utf-8") as file:                                                
            write = csv.writer(file)
            write.writerow(x_names)
            write.writerows(n.X_train)            
        
        with open("./cross_validation_files/target_validation_train_{}_{}.csv".format(str(k), str(c)), 'w', newline='', encoding="utf-8") as file:                                                
            write = csv.writer(file)
            write.writerow(y_names)
            write.writerows([[i] for i in n.y_train])

        with open("./cross_validation_files/data_test_{}_{}.csv".format(str(k), str(c)), 'w', newline='', encoding="utf-8") as file:                                                
            write = csv.writer(file)
            write.writerow(x_names)
            write.writerows(n.X_test)

        with open("./cross_validation_files/target_test_{}_{}.csv".format(str(k), str(c)), 'w', newline='', encoding="utf-8") as file:                                                
            write = csv.writer(file)
            write.writerow(y_names)
            write.writerows([[i] for i in n.y_test])   
    
    with open("./cross_validation_files/data_test.csv", 'w', newline='', encoding="utf-8") as file:                                                
            write = csv.writer(file)
            write.writerow(x_names)
            write.writerows(my_data_set.test_set.X_test)    
              
    y_test = [[i] for i in my_data_set.test_set.y_test]
    with open("./cross_validation_files/target_test.csv", 'w', newline='', encoding="utf-8") as file:                                                
            write = csv.writer(file)
            write.writerow(y_names)
            write.writerows(y_test) 

In [16]:
def main(ks):
    for k in ks:
        data = generate_train_test(data_path, k)

        #Guardar archivo pkl
        file = open('./cross_validation_files/data_set_k-{}.pkl'.format(str(k)), 'wb')
        pickle.dump(data, file)
        file.close()

        createFiles(data, k)    
    print("Se ha concluido el proceso corectamente")
#Lista de pliegues 
main([3, 5, 10])

Se ha concluido el proceso corectamente
